In [ ]:
import torch
from torch import nn, optim
import os 
import torch.nn.functional as F

In [ ]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("encoder/saved_models/pretrained.pt")
encoder.load_model(encoder_weights)
vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)


In [ ]:
#   in_fpath = Path("voicesrc\\Pan1.wav")
# subdir='id10286-isKyMAYUOgg-00004'
filepath=r'D:\voiceset\dataset\test_wav'
in_fpath=os.path.join(filepath,subdir[:7],subdir[8:-6],subdir[-5:]+'.wav')
#读取文件
file=open('..\I2E\ivectortest.txt')
file.readlines()

In [ ]:
def embedEX(in_fpath):
    filepath=r'D:\voiceset\dataset\test_wav'
#     filepath=r'D:\voiceset\dataset\wav'
    in_fpath=os.path.join(filepath,in_fpath[:7],in_fpath[8:-6],in_fpath[-5:]+'.wav')
    reprocessed_wav = encoder.preprocess_wav(in_fpath)
    original_wav, sampling_rate = librosa.load(in_fpath)
    preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
    embed = encoder.embed_utterance(preprocessed_wav)
    return embed

In [ ]:
class I2E_Net(torch.nn.Module):
    def __init__(self, in_dim, n_hidden_1, out_dim):
        super(I2E_Net, self).__init__()
        self.layer1 = torch.nn.Sequential(torch.nn.Linear(in_dim, n_hidden_1), torch.nn.ReLU(True))
        self.layer2 = torch.nn.Sequential(torch.nn.Linear(n_hidden_1, out_dim))
#         self.layer2 = torch.nn.Sequential(torch.nn.Linear(n_hidden_1, n_hidden_2), torch.nn.ReLU(True))
#         self.layer3 = torch.nn.Sequential(torch.nn.Linear(n_hidden_2, out_dim))
 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
#         x = self.layer3(x)
        return x

In [ ]:
class I2E_Net(torch.nn.Module):
    def __init__(self, in_dim, n_hidden_1,n_hidden_2,out_dim):
        super(I2E_Net2, self).__init__()
        self.layer1 = torch.nn.Sequential(torch.nn.Linear(in_dim, n_hidden_1), torch.nn.ReLU(True))
        self.layer2 = torch.nn.Sequential(torch.nn.Linear(n_hidden_1, n_hidden_2), torch.nn.ReLU(True))
        self.layer3 = torch.nn.Sequential(torch.nn.Linear(n_hidden_2, out_dim))
 
    def forward(self, x):
        x=x/torch.norm(x, p=2)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x=x/torch.norm(x, p=2)
        return x

In [ ]:
class I2E_Net2MSE(torch.nn.Module):
    def __init__(self, in_dim, n_hidden_1,n_hidden_2,out_dim):
        super(I2E_Net2, self).__init__()
        self.layer1 = torch.nn.Sequential(torch.nn.Linear(in_dim, n_hidden_1), torch.nn.ReLU(True))
        self.layer2 = torch.nn.Sequential(torch.nn.Linear(n_hidden_1, n_hidden_2), torch.nn.ReLU(True))
        self.layer3 = torch.nn.Sequential(torch.nn.Linear(n_hidden_2, out_dim))
 
    def forward(self, x):
        x=x/torch.norm(x, p=2)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x=x/torch.norm(x, p=2)
        return x

In [ ]:
class I2E_Net3(torch.nn.Module):
    def __init__(self):
        super(I2E_Net3, self).__init__()
        self.layer1 = torch.nn.Sequential(nn.Conv1d(400,350,1), nn.ReLU())
        self.layer2 = torch.nn.Sequential(nn.Conv1d(350,300,1), nn.ReLU())
#         self.layer3 = torch.nn.Sequential(nn.Conv1d(300,256,1), nn.ReLU())
        self.fc = torch.nn.Linear(300,256)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
class I2E_Net5(torch.nn.Module):
    def __init__(self):
        super(I2E_Net5, self).__init__()
        self.layer1 = torch.nn.Sequential(nn.Conv2d(1,1,1)), nn.ReLU())
        self.layer2 = torch.nn.Sequential(nn.Conv2d(1,1,(1,9))), nn.ReLU())
        self.layer2 = torch.nn.Sequential(nn.Conv2d(1,1,(1,9))), nn.ReLU())
        self.layer2 = torch.nn.Sequential(nn.Conv2d(1,1,(1,9))), nn.ReLU())
        self.layer2 = torch.nn.Sequential(nn.Conv2d(1,1,(1,9))), nn.ReLU())
#         self.layer3 = torch.nn.Sequential(nn.Conv1d(300,256,1), nn.ReLU())0
        self.fc = torch.nn.Linear(300,256)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
class I2E_Net4(torch.nn.Module):
    def __init__(self):
        super(I2E_Net4, self).__init__()
#         ,bidirectional=True
        self.lstm = torch.nn.LSTM(400,256,1)
        self.linear=torch.nn.Linear(256,256)
        self.relu=torch.nn.ReLU()
    def forward(self, x):
        out,(hidden,cell)= self.lstm(x)
#         x= self.relu( self.linear(hidden[-1]))
        x= self.relu( self.linear(hidden[-1]))
        x=x/torch.norm(x,dim=1,keepdim=True)
        return x

In [ ]:
# 定义一些超参数
batch_size = 64
learning_rate = 0.02
num_epoches = 20

In [ ]:
#模型
model=I2E_Net2MSE(400,350,300,256)
#损失函数
# criterion = torch.nn.MSELoss(reduce=True, size_average=False)
criterion = torch.nn.MSELoss(reduction='sum')
# criterion = torch.nn.L1Loss(reduction='sum')
#优化器
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 模型
# model=I2E_Net3()
#损失函数0,
model=I2E_Net4()
# criterion = torch.nn.MSELoss(reduce=True, size_average=False) reduction='sum'
criterion = torch.nn.MSELoss(reduction='sum')
#优化器
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 训练模型
epoch = 0
totaldir='D:\\pycharm_project\\vs2ttsvoice\\I2E'
#损失值写入文件

for i in range(80):
    pathstr =str(i+1)+'.txt' 
    pathstr=os.path.join(totaldir, pathstr)
    file=open( pathstr)
    for line in file.readlines():
        data=line.strip().replace('[','').replace(']','').split()
        filename=data[0]
        data=data[1:]
#         out=[[list(map(float,data))]]
#         out=torch.tensor(out)
#         out=out.permute(0,2,1)
#         pred=model(out)
#         pred=pred.squeeze(0)
        out=list(map(float,data))
        out=torch.tensor(out)
        pred=model(out)
        label=embedEX(filename)
        label=torch.from_numpy(label)
        loss = criterion(pred, label)
        print_loss = loss.data.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch+=1
        if epoch%100 == 0:
        #         print('label:{0}\n,out:{1}\n,pred:{2}\n'.format(label.size(),out.size(),pred.size()))
        #保存参数、
            print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))
            state = { 'model':model.state_dict(), 'epoch':epoch }                       
            if not os.path.isdir('checkpoint'):
                 os.mkdir('checkpoint')    
            torch.save(state, '.\checkpoint\model4_epoch_%d.ckpt' % (epoch))
            file_save = open('loss2.txt',mode='a')
            file_save.write('\n'+'epoch:'+str(epoch)+'  loss:'+str(loss.data.item()))
            file_save.close() 

In [ ]:
# 训练模型3,将参数写入文件中
epoch = 0
totaldir=r"D:\pycharm_project\vs2ttsvoice\I2E"
#损失值写入文件

for i in range(80):
    pathstr =str(i+1)
    pathstr=os.path.join(totaldir, pathstr)
    file=open( pathstr)
    for line in file.readlines():
        data=line.strip().replace('[','').replace(']','').split()
        filename=data[0]
        data=data[1:]
        out=[[list(map(float,data))]]
        out=torch.tensor(out)
        out=out.permute(0,2,1)
        pred=model(out)
        pred=pred.squeeze(0)
        label=embedEX(filename)
        label=torch.from_numpy(label)
        loss = criterion(pred, label)
        print_loss = loss.data.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch+=1
        if epoch%100 == 0:
        #         print('label:{0}\n,out:{1}\n,pred:{2}\n'.format(label.size(),out.size(),pred.size()))
        #保存参数、
            print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))
            state = { 'net':model.state_dict(), 'epoch': epoch, }                                 # 1 、 先建立一个字典
            if not os.path.isdir('checkpoint'):
                 os.mkdir('checkpoint')       # 2 、 建立一个保存参数的文件夹
            torch.save(state, './checkpoint/epoch_%d.ckpt' % (epoch))
            #保存损失函数
# file_save = open('loss.txt','w')
# file_save.write('\n'+'epoch:'+str(epoch)+'  loss:'+loss.data.item())
# file_save.close() 

In [ ]:
# 训练模型4,将参数写入文件中
epoch = 0
totaldir=r"D:\pycharm_project\vs2ttsvoice\I2E"
#损失值写入文件

for i in range(80):
    pathstr =str(i+1)+'.txt'
    pathstr=os.path.join(totaldir, pathstr)
    file=open( pathstr)
    for line in file.readlines():
        data=line.strip().replace('[','').replace(']','').split()
        filename=data[0]
        data=data[1:]
        out=[[list(map(float,data))]]
        out=torch.tensor(out)
        pred=model(out)
        pred=pred.squeeze(0)
        label=embedEX(filename)
        label=torch.from_numpy(label)
        loss = criterion(pred, label)
        print_loss = loss.data.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch+=1
        if epoch%100 == 0:
        #         print('label:{0}\n,out:{1}\n,pred:{2}\n'.format(label.size(),out.size(),pred.size()))
        #保存参数、
            print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))
#             state = { 'net':model.state_dict(), 'epoch': epoch, }                                 # 1 、 先建立一个字典
#             if not os.path.isdir('checkpoint'):
#                  os.mkdir('checkpoint')       # 2 、 建立一个保存参数的文件夹
#             torch.save(state, './checkpoint/epoch_%d.ckpt' % (epoch))

In [ ]:
# 训练模型
epoch = 0
file=open('..\I2E\ivectortest.txt')
for line in file.readlines():
    data=line.strip().replace('[','').replace(']','').split()
    filename=data[0]
    data=data[1:]
    out=[[list(map(float,data))]]
    out=torch.tensor(out)
    out=out.permute(0,2,1)
    pred=model(out)
    pred=pred.squeeze(0)
    label=embedEX(filename)
    label=torch.from_numpy(label)
    loss = criterion(pred, label)
    print_loss = loss.data.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    epoch+=1
    print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))
#     if epoch%50 == 0:
#         print('label:{0}\n,out:{1}\n,pred:{2}\n'.format(label.size(),out.size(),pred.size()))
#         print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))

In [ ]:
in_fpath=r'D:\voiceset\dataset\test_wav\id10270\5r0dWxy17C8\00003.wav'
reprocessed_wav = encoder.preprocess_wav(in_fpath)
original_wav, sampling_rate = librosa.load(in_fpath)
preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
embed1 = encoder.embed_utterance(preprocessed_wav)

In [ ]:
in_fpath=r'D:\voiceset\dataset\test_wav\id10270\5r0dWxy17C8\00004.wav'
reprocessed_wav = encoder.preprocess_wav(in_fpath)
original_wav, sampling_rate = librosa.load(in_fpath)
preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
embed2 = encoder.embed_utterance(preprocessed_wav)

In [ ]:
# 读取训练参数
# checkpoint = torch.load(r'C:\NLP\tts_rtvc\rtvc\checkpoint\model3_epoch_56300.ckpt')
checkpoint = torch.load(r'D:\pycharm_project\vs2ttsvoice\checkpoint\model4_epoch_98800.ckpt')
model.load_state_dict(checkpoint['model'])

In [ ]:
import soundfile as sf

In [ ]:
#合成语音
from IPython.utils import io
epoch=0
text="This is being said in my own voice.  The computer has learned to do an impression of me."
file=open('.\I2E\ivectortest.txt')
filenamelabel=''
for line in file.readlines():
# line=file.readline()
    data=line.strip().replace('[','').replace(']','').split()
    filename=data[0]
    if (filenamelabel==filename[:7]):
        continue
    print(filename)
    data=data[1:]
#     out=[[list(map(float,data))]]
#     out=torch.tensor(out)
#     out=out.permute(0,2,1)
#     pred=model(out)
#     pred=pred.squeeze(0)
#     pred=pred.tolist()
#     print("pred",pred)
    out=list(map(float,data))
    out=torch.tensor(out)
    pred=model(out)
    pred=pred.tolist()
    print("pred",pred)
    print("Synthesizing 预测语音...")
    with io.capture_output() as captured:
      specs = synthesizer.synthesize_spectrograms([text], [pred])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
    sf.write(filename[:7]+"A.wav", generated_wav, synthesizer.sample_rate)
    epoch =+1
    #原始声音合成
    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
    label=embedEX(filename)
    print("Synthesizing 原始语音...")
    print('阈值：',np.linalg.norm(pred-label))
    with io.capture_output() as captured:
       specs = synthesizer.synthesize_spectrograms([text], [label])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
    filenamelabel=filename[:7]
    sf.write(filename[:7]+".wav", generated_wav, synthesizer.sample_rate)

In [ ]:
#合成测试集所有的语音
from IPython.utils import io
epoch=0
text="This is being said in my own voice.The computer has learned to do an impression of me."
file=open('.\I2E\ivectortest.txt')
filesynVS=r'D:\voiceset\dataset\testVS_wav'
filesynI2E=r'D:\voiceset\dataset\testI2E_wav'
filenamelabel=''
for line in file.readlines():
# line=file.readline()
    data=line.strip().replace('[','').replace(']','').split()
    filename=data[0]
    #建立VS目录in_fpath=os.path.join(filepath,in_fpath[:7],in_fpath[8:-6],in_fpath[-5:]+'.wav')
    pathvs=os.path.join(filesynVS,filename[:7],filename[8:-6])
    if not os.path.exists(pathvs):
        os.makedirs(pathvs)
        
    #创建i2目录    
    pathi2e=os.path.join(filesynI2E,filename[:7],filename[8:-6])        
    if not os.path.exists(pathi2e):
        os.makedirs(pathi2e)

    data=data[1:]
    out=list(map(float,data))
    out=torch.tensor(out)
    pred=model(out)
    pred=pred.tolist()
#     print("pred",pred)
    print("Synthesizing 预测语音...")
    with io.capture_output() as captured:
      specs = synthesizer.synthesize_spectrograms([text], [pred])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
    
    sf.write(os.path.join(pathi2e,filename[-5:]+'.wav'), generated_wav, synthesizer.sample_rate)
    epoch =+1
    #原始声音合成
    print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
    label=embedEX(filename)
    print("Synthesizing 原始语音...")
    print('阈值：',np.linalg.norm(pred-label))
    with io.capture_output() as captured:
       specs = synthesizer.synthesize_spectrograms([text], [label])
    generated_wav = vocoder.infer_waveform(specs[0])
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    display(Audio(generated_wav, rate=synthesizer.sample_rate))
    filenamelabel=filename[:7]
    sf.write(os.path.join(pathvs,filename[-5:]+'.wav'), generated_wav, synthesizer.sample_rate)